# Analysis

In [1]:
import os
import pandas as pd
from importlib import reload

# Get the current working directory
os.chdir("c:/Users/juanm_8qa8lav/Documents/Proyectos_Personales/FinancialApp")
# Verify the current working directory
print(f"Now in directory: {os.getcwd()}")

# Import your module
from app.modules.database.connection import engine
from app.modules.database.reports import pp, customers, credits, collections, installments, companies, bp, companies, portfolio_inventory
from app.modules.database.credit_manager import credits_balance
from app.modules.database.portfolio_manager import portfolio_seller

balance = credits_balance()
balance['D_Due'] = balance['D_Due'].dt.to_period('D')

print(f'Capital: $ {balance['Capital'].sum():,.2f}')
print(f'Interés: $ {balance['Interest'].sum():,.2f}')
print(f'IVA: $ {balance['IVA'].sum():,.2f}')
print(f'Total: $ {balance['Total'].sum():,.2f}')

Now in directory: c:\Users\juanm_8qa8lav\Documents\Proyectos_Personales\FinancialApp
Capital: $ 34,891,606.64
Interés: $ 31,843,591.08
IVA: $ 6,687,154.04
Total: $ 73,422,351.89


In [2]:
date = pd.Period.now('D')

tna = 0.6
va = 20e6

inst, crts, cust  = portfolio_seller(date, tna, va, 2, save=False)
# Falta armar los reportes de la venta. 

TIR: 4.33%
